 # Lab 6



 Grover's Search

 ## Task 1


 Simple Password Cracking with Grover's Algorithm


 Scenario: We have a 3-bit password (0-7) and we know its hash.

 We want to find which password produces that hash.


 For simplicity, we'll use a toy hash function:

 `hash(x) = (3*x + 5) mod 8` 


 Target: Find x where hash(x) = 5

 Answer: hash(0) = (3*0 + 5) mod 8 = 5, so password is 0

In [ ]:
def toy_hash(x):
    """Hash function with unique preimages: (3*x + 5) mod 8."""
    return (3 * x + 5) % 8



In [ ]:
# Setup
n_qubits = 3  # 3-bit password (values 0-7)
target_hash = 5  # The hash value we want to find a pre-image for



In [ ]:
def classical_preimage_search():
    """Classically search for a pre-image of the target hash."""
    for x in range(8):
        print(f"h({x}) = {toy_hash(x)}")
        if toy_hash(x) == target_hash:
            return x
    return None


actual_password = classical_preimage_search()



In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit.library import MCMTGate, ZGate
from qiskit_aer import AerSimulator
import numpy as np


In [ ]:
def create_oracle(n_qubits, target_hash):
    """
    Create an oracle that marks states x where hash(x) == target_hash.
    Implements: hash(x) = (3*x + 5) mod 8
    """

    # n_qubits for input, n_qubits for hash output, 1 ancilla for phase kickback
    total_qubits = n_qubits + n_qubits + 1
    oracle = QuantumCircuit(total_qubits, name="Oracle")

    input_qubits = list(range(n_qubits))
    hash_qubits = list(range(n_qubits, n_qubits + n_qubits))
    phase_qubit = 2 * n_qubits

    # ---------------------------------------------------------------------
    # Step 1: Compute hash(x) = (3*x + 5) mod 8
    # ---------------------------------------------------------------------
    # We'll do it as: hash = 2*x + x + 5
    #
    # 2*x means a left shift of bits (x1 -> hash2, x0 -> hash1, etc.)
    # For 3 bits: 2*x mod 8 just rotates left by one bit and discards overflow.

    # Copy x to hash (so hash = x)
    for i in range(n_qubits):
        oracle.cx(input_qubits[i], hash_qubits[i])

    # Add 2*x (left shift by 1 bit)
    for i in range(n_qubits - 1):
        oracle.cx(input_qubits[i], hash_qubits[i + 1])
    # overflow bit (input_qubits[n_qubits-1]) discarded since mod 8

    # Add 5 (binary 101)
    # 5 = 0b101 -> flip hash_qubits[0] and hash_qubits[2]
    oracle.x(hash_qubits[0])
    if n_qubits >= 3:
        oracle.x(hash_qubits[2])

    # ---------------------------------------------------------------------
    # Step 2: Check if hash(x) == target_hash
    # ---------------------------------------------------------------------
    target_binary = format(target_hash, f"0{n_qubits}b")

    # Flip qubits where target bit is 0 so we match on |111...>
    for i, bit in enumerate(target_binary):
        if bit == "0":
            oracle.x(hash_qubits[i])

    # Multi-controlled X on the phase qubit
    oracle.mcx(hash_qubits, phase_qubit)

    # Unflip the bits
    for i, bit in enumerate(target_binary):
        if bit == "0":
            oracle.x(hash_qubits[i])

    # ---------------------------------------------------------------------
    # Step 3: Uncompute the hash to clean up ancilla
    # ---------------------------------------------------------------------
    # Reverse steps of Step 1 (uncompute 5, then 2*x, then x)

    # Undo add 5
    oracle.x(hash_qubits[0])
    if n_qubits >= 3:
        oracle.x(hash_qubits[2])

    # Undo 2*x addition
    for i in reversed(range(n_qubits - 1)):
        oracle.cx(input_qubits[i], hash_qubits[i + 1])

    # Undo copy x
    for i in reversed(range(n_qubits)):
        oracle.cx(input_qubits[i], hash_qubits[i])

    return oracle

In [ ]:

def create_diffusion(n_qubits):
    """Create diffusion operator (inversion about average)."""
    diffusion = QuantumCircuit(n_qubits, name="Diffusion")

    # Step 1: Apply H to all qubits
    diffusion.h(range(n_qubits))

    # Step 2: Apply X to all qubits
    diffusion.x(range(n_qubits))

    # Step 3: Multi-controlled Z gate
    # This flips the phase of |00...0>, which after the X gates corresponds to |11...1>
    # Implemented as H–MCX–H on the last qubit
    diffusion.h(n_qubits - 1)
    diffusion.mcx(list(range(n_qubits - 1)), n_qubits - 1)
    diffusion.h(n_qubits - 1)

    # Step 4: Apply X to all qubits
    diffusion.x(range(n_qubits))

    # Step 5: Apply H to all qubits
    diffusion.h(range(n_qubits))

    return diffusion


In [ ]:
# Calculate number of Grover iterations
N = 2**n_qubits
optimal_iterations = int(np.pi / 4 * np.sqrt(N)) # Since there's 1 solution, otherwise use N/M, where M is the number of solutions


In [ ]:
# Assuming n_qubits, target_hash, and optimal_iterations are defined earlier
total_qubits = n_qubits + n_qubits + 1  # Input + hash + ancilla
qr = QuantumRegister(total_qubits, "q")
cr = ClassicalRegister(n_qubits, "c")  # Only measure the input qubits

qc = QuantumCircuit(qr, cr)

input_qubits = list(range(n_qubits))
hash_qubits = list(range(n_qubits, 2 * n_qubits))
phase_qubit = 2 * n_qubits

# ----------------------------------------------------------------------
# Step 1: Initialize input qubits to superposition (|+> states)
# ----------------------------------------------------------------------
qc.h(input_qubits)

# ----------------------------------------------------------------------
# Step 2: Initialize phase qubit to |-⟩ = (|0⟩ - |1⟩)/√2
# ----------------------------------------------------------------------
qc.x(phase_qubit)
qc.h(phase_qubit)

qc.barrier()

# ----------------------------------------------------------------------
# Step 3: Grover iterations (Oracle + Diffusion)
# ----------------------------------------------------------------------
oracle = create_oracle(n_qubits, target_hash)
diffusion = create_diffusion(n_qubits)

for _ in range(optimal_iterations):
    # Apply oracle on all relevant qubits
    qc.append(oracle.to_gate(), qr[:])
    qc.barrier()
    # Apply diffusion on input qubits only
    qc.append(diffusion.to_gate(), input_qubits)
    qc.barrier()

# ----------------------------------------------------------------------
# Step 4: Measure input qubits
# ----------------------------------------------------------------------
qc.measure(input_qubits, cr)

In [ ]:
# Simulate the circuit
simulator = AerSimulator()
job = simulator.run(transpile(qc, simulator), shots=4096)
result = job.result()
counts = result.get_counts()

sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
for state, count in sorted_counts:
    percentage = count / 4096 * 100
    password_guess = int(state, 2)
    hash_value = toy_hash(password_guess)
    marker = "✓✓✓" if password_guess == actual_password else ""
    print(
        f"  {state} (password={password_guess}): {count:4d} shots ({percentage:5.1f}%) - h({password_guess})={hash_value} {marker}"
    )

# Find the most likely password guess
most_common = max(counts, key=counts.get)
found_password = int(most_common, 2)

print(
    f"\nMost likely password guess: {found_password} (h({found_password})={toy_hash(found_password)})"
)
